### Code to Rename all FIles in the given Folder in drive

In [10]:
# import os
# import shutil

# folder_path = r'./Evaluation_results/Two_Model_ROC_curves'  # Specify the folder path here

# for root, dirs, files in os.walk(folder_path):
#     for dir_name in dirs:
#         new_dir_name = dir_name.replace(' ', '_')
#         old_dir_path = os.path.join(root, dir_name)
#         new_dir_path = os.path.join(root, new_dir_name)
#         shutil.move(old_dir_path, new_dir_path)

#     for file_name in files:
#         new_file_name = file_name.replace(' ', '_')
#         old_file_path = os.path.join(root, file_name)
#         new_file_path = os.path.join(root, new_file_name)
#         shutil.move(old_file_path, new_file_path)

In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

In [2]:
# !pip install python-docx

### Code to write a custom word file out of images in the given folder

In [3]:
import os
from PIL import Image
from docx import Document
from docx.shared import Inches, Pt
import re
from docx.oxml.ns import nsdecls
from docx.oxml import parse_xml
from docx.enum.text import WD_PARAGRAPH_ALIGNMENT

def convert_png_to_jpeg(image_path):
    image = Image.open(image_path).convert("RGB")
    jpeg_path = os.path.splitext(image_path)[0] + ".jpg"
    image.save(jpeg_path, "JPEG")
    return jpeg_path

def extract_experiment_and_image_numbers(filename):
    match = re.match(r'(\d+)\.(\d+)_', filename)
    if match:
        experiment_number = int(match.group(1))
        image_number = match.group(2).zfill(2)  # Pad image number with leading zeros
        return experiment_number, image_number
    return 0, "00"  # Default values to handle filenames without experiment and image numbers

def custom_sort(image_files):
    return sorted(image_files, key=lambda x: (extract_experiment_and_image_numbers(x)[0], extract_experiment_and_image_numbers(x)[1]))


def create_word_from_images(folder_path, output_path,new_width, new_height):
    image_files = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]
    image_files = custom_sort(image_files)
#     image_files.sort(key=lambda x: (extract_experiment_and_image_numbers(x)[0], extract_experiment_and_image_numbers(x)[1]))
#     image_files.sort(key=lambda x: extract_experiment_and_image_numbers(x))

    document = Document()
    count = 0
    # Define custom paragraph style for the image titles
    title_style_xml = """
        <w:style xmlns:w="http://schemas.openxmlformats.org/wordprocessingml/2006/main" w:type="paragraph" w:styleId="ImageTitle">
            <w:name w:val="Image Title" />
            <w:basedOn w:val="Normal" />
            <w:qFormat />
            <w:rsid w:val="00" />
            <w:pPr>
                <w:spacing w:line="240" w:lineRule="auto" />
                <w:jc w:val="center" />
            </w:pPr>
            <w:rPr>
                <w:rFonts w:asciiTheme="minorHAnsi" w:eastAsiaTheme="minorHAnsi" w:hAnsiTheme="minorHAnsi" w:cstheme="minorBidi" />
                <w:sz w:val="20" />
                <w:szCs w:val="20" />
                <w:lang w:eastAsia="en-US" />
            </w:rPr>
        </w:style>
    """
    style_element = parse_xml(title_style_xml)
    document.styles.element.append(style_element)

#     new_width = 100
#     new_height = 100

    for image_file in image_files:
        count+=1
        print(count, image_file)
        image_path = os.path.join(folder_path, image_file)

#         if image_path.lower().endswith(".png"):
#             # Convert PNG image to JPEG
#             image_path = convert_png_to_jpeg(image_path)
        
        # Create a new paragraph for the image
        paragraph = document.add_paragraph()
        paragraph.alignment = 1  # Set paragraph alignment to center
        # Add the image to the paragraph
        run = paragraph.add_run()        
        # Add the image to the Word document with resizing
        section = document.sections[-1]
        page_width = section.page_width - section.left_margin - section.right_margin
        page_height = section.page_height - section.top_margin - section.bottom_margin
        print('##########PAGE_WIDTH########', page_width, page_height)
        if ('Thumbs.db' not in image_path):
            image = Image.open(image_path)
            image_width, image_height = image.size
            print('##########IMAGE_WIDTH########', image_width, image_height)
            if image_width*1.5 > page_width/10000 or image_height > page_height/10000:
                # Resize the image to fit the page
                width_ratio = page_width/10000 / image_width / 1.79
                height_ratio = page_height/10000 / image_height / 1.79
                print('WIDTH AND HEIGHT RATIO ------',width_ratio,height_ratio)
                resize_ratio = min(width_ratio, height_ratio)
                new_width = int(image_width * resize_ratio)
                new_height = int(image_height * resize_ratio)
                image = image.resize((new_width, new_height))
                run.add_picture(image_path, width=Inches(new_width / 72), height=Inches(new_height / 72))#, align='center')            
    #             document.add_picture(image_path, width=Inches(new_width / 72), height=Inches(new_height / 72))
            else:
                run.add_picture(image_path, width=Inches(new_width / 72), height=Inches(new_height / 72))#, align='center')
    #             document.add_picture(image_path, width=Inches(image_width / 72), height=Inches(image_height / 72))

            # Add the image filename as the title of the image with the custom style
            title_paragraph = document.add_paragraph(style='ImageTitle')
            title_paragraph.alignment = 1  # Center alignment
            image_file = os.path.splitext(image_file)[0]
            # For teacher and student models and ensembles
            title_run = title_paragraph.add_run('Fig:'+str(image_file.split("_")[0])+" Model:"+str("_".join(image_file.split("_")[1:-2]))+" | Metrics: "+ str(image_file.split("_")[-2])+" | Dataset: "+ str(image_file.split("_")[-1]))
            # For two model curves
#             title_run = title_paragraph.add_run('Fig:'+str(image_file.split("_")[0])+" UndistilledStudent:"+str(image_file.split("_")[1])+" Teacher:"+str(image_file.split("_")[3])+" | Metrics: "+ str(image_file.split("_")[-2])+" | Dataset: "+ str(image_file.split("_")[-1]))
            title_run.font.size = Pt(9) # Font size 9
            title_run.font.underline = False # Remove underline

            # Adjust paragraph alignment to center
            paragraph_format = title_paragraph.paragraph_format
            paragraph_format.alignment = WD_PARAGRAPH_ALIGNMENT.CENTER
    # Save the Word document
    document.save(output_path)

# Usage example:
width = 20
height = 20
# folder_path = './Evaluation_results/Possible_Teacher_Models/'
# output_path = './Evaluation_results/Possible_Teacher_Models.docx'
# folder_path = './Evaluation_results/Possible_Student_Models/'
# output_path = './Evaluation_results/Possible_Student_Models.docx'
# folder_path = './Evaluation_results/Possible_Teacher_Ensembles/'
# output_path = './Evaluation_results/Possible_Teacher_Ensembles.docx'
folder_path = './Evaluation_results/Distilled_Students/'
output_path = './Evaluation_results/Distillation_Experiments.docx'
# folder_path = './Evaluation_results/Two_Model_ROC_curves/'
# output_path = './Evaluation_results/Two_Model_ROC_curves.docx'
create_word_from_images(folder_path, output_path, width, height)

1 1.1_MobileNetv3SmallMinimalisticDistilled_AccuracyQwkAuc_IDRID.png
##########PAGE_WIDTH######## 5486400 8229600
##########IMAGE_WIDTH######## 569 68
WIDTH AND HEIGHT RATIO ------ 0.5386692325063083 6.761091028590207
2 1.2_MobileNetv3SmallMinimalisticDistilled_ConfMatrix_IDRID.png
##########PAGE_WIDTH######## 5486400 8229600
##########IMAGE_WIDTH######## 600 398
WIDTH AND HEIGHT RATIO ------ 0.5108379888268156 1.1551612812666685
3 1.3_MobileNetv3SmallMinimalisticDistilled_ROCCurve_IDRID.png
##########PAGE_WIDTH######## 5486400 8229600
##########IMAGE_WIDTH######## 567 453
WIDTH AND HEIGHT RATIO ------ 0.5405693003458366 1.014909911576455
4 1.4_MobileNetv3SmallMinimalisticDistilled_AccuracyQwkAuc_Kaggle.png
##########PAGE_WIDTH######## 5486400 8229600
##########IMAGE_WIDTH######## 567 64
WIDTH AND HEIGHT RATIO ------ 0.5405693003458366 7.1836592178770955
5 1.5_MobileNetv3SmallMinimalisticDistilled_ConfMatrix_Kaggle.png
##########PAGE_WIDTH######## 5486400 8229600
##########IMAGE_WIDTH#

/home/user/.local/lib/python3.9/site-packages/docx/styles/styles.py:139: UserWarning: style lookup by style_id is deprecated. Use style name as key instead.
  return self._get_style_id_from_style(self[style_name], style_type)


13 2.4_MobileNetv3SmallMinimalisticDistilled_AccuracyQwkAuc_Kaggle.png
##########PAGE_WIDTH######## 5486400 8229600
##########IMAGE_WIDTH######## 573 68
WIDTH AND HEIGHT RATIO ------ 0.5349088888238906 6.761091028590207
14 2.5_MobileNetv3SmallMinimalisticDistilled_ConfMatrix_Kaggle.png
##########PAGE_WIDTH######## 5486400 8229600
##########IMAGE_WIDTH######## 600 398
WIDTH AND HEIGHT RATIO ------ 0.5108379888268156 1.1551612812666685
15 2.6_MobileNetv3SmallMinimalisticDistilled_ROCCurve_Kaggle.png
##########PAGE_WIDTH######## 5486400 8229600
##########IMAGE_WIDTH######## 567 453
WIDTH AND HEIGHT RATIO ------ 0.5405693003458366 1.014909911576455
16 2.7_MobileNetv3SmallMinimalisticDistilled_AccuracyQwkAuc_Messidor2.png
##########PAGE_WIDTH######## 5486400 8229600
##########IMAGE_WIDTH######## 571 71
WIDTH AND HEIGHT RATIO ------ 0.5367824751244997 6.47541112597372
17 2.8_MobileNetv3SmallMinimalisticDistilled_ConfMatrix_Messidor2.png
##########PAGE_WIDTH######## 5486400 8229600
##########